In [ ]:
#from jupyter_dash.comms import _send_jupyter_config_comm_request
#_send_jupyter_config_comm_request()

In [53]:
# Import `app` object from `display.py` module
from displayy import app
from jupyter_dash import JupyterDash  # noQA F401

#JupyterDash.infer_jupyter_proxy_config()

# Start app server
app.run_server(host="localhost", port=3000, mode='external', debug =True, threaded=True)

Dash app running on http://localhost:3000/


In [1]:
import plotly.express as px
import pandas as pd
df = pd.DataFrame(dict(
    x = [1, 3, 2, 4],
    y = [1, 2, 3, 4]
))

In [8]:
type(df)

pandas.core.frame.DataFrame

In [39]:
import sqlite3
from data import SQLRepository
from data import AlphaVantageApi
from config import settings

connection = sqlite3.connect(settings.db_name, check_same_thread=False)
api = AlphaVantageApi()
repo = SQLRepository(connection=connection)

In [127]:
ti = "ADP"
("SELECT name FROM sqlite_master WHERE name= '%s' " %ti)

"SELECT name FROM sqlite_master WHERE name= 'ADP' "

In [109]:
("""SELECT name FROM sqlite_master WHERE name='ADP' """)

"SELECT name FROM sqlite_master WHERE name='ADP' "

In [129]:

#data = repo.read_table("TYU")
ti="ADP"
res = connection.execute("SELECT name FROM sqlite_master WHERE name= '%s' " %ti)
if res.fetchone() != None:
    data = repo.read_table(table_name='ADP')
    if data.index.max() == pd.to_datetime('today').normalize():
        data = data
        #data = api.
else:
    data = api.get_historical_data(ticker="ADP")
    repo.insert_table(table_name=ticker, records=data, if_exists="Replace")
 
data.head()

,open,high,low,close
date,,,,
2023-03-03,223.120,225.130,222.750,224.75
2023-03-02,219.815,223.250,219.350,222.55
2023-03-01,220.150,220.880,217.815,219.56
2023-02-28,220.910,222.265,219.400,219.82
2023-02-27,223.000,224.235,221.170,221.60


In [61]:
import json
import plotly.graph_objects as go
fig = px.line(data, x=data.index, y='close', title = f"Stock Historical Price")
fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
dict_fig = fig.to_dict()
fig=go.Figure(dict_fig)
fig.show()
#json.dumps(dict_fig, indent = 2)

In [46]:
data.sort_index(ascending = True, inplace = True)
data["return"] = data["close"].pct_change() * 100
data = data.dropna()
value = "Price"
if value == "Volatility":
    data['rolling_6d_volatility'] = data['return'].rolling(window=6).std().dropna()
    fig = build_graph(data, 'rolling_6d_volatility')
    fig.show()
else:
    fig = px.line(data, x=data.index, y='close', title = f"Stock Historical Price")
    fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
    fig.show()


In [52]:
data.iloc[:,5]

date
1999-11-12    1.182234
1999-11-15    1.037109
1999-11-16    1.249570
1999-11-17    1.104649
1999-11-18    1.367101
                ...   
2023-02-27    0.793038
2023-02-28    0.640086
2023-03-01    0.369052
2023-03-02    0.748919
2023-03-03    0.853981
Name: rolling_6d_volatility, Length: 5863, dtype: float64

In [34]:
pd.to_datetime("today").normalize()

Timestamp('2023-03-04 00:00:00')

In [67]:
import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository
import plotly.graph_objects as go

In [158]:
#check if the table exists in the database
ti = "AAL"
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)
res = connection.execute("SELECT name FROM sqlite_master WHERE name= '%s' " %ti)
if res.fetchone() != None:
#if the table exist, check if the data is up to date
    
    data = repo.read_table(table_name=ti, limit=2500+1)
    if data.index.max() == pd.to_datetime('today').normalize():
        data = data
    else:
        #Download a new data using the ticker
        api = AlphaVantageApi()
        data = api.get_historical_data(ticker=ti)
        repo.insert_table(table_name = ti, records = data, if_exists="replace")
        data = repo.read_table(table_name=ti, limit=2500 + 1)
else:
    api = AlphaVantageApi()
    data = api.get_historical_data(ticker=ti)
    repo.insert_table(table_name = ti, records = data, if_exists="replace")
    data = repo.read_table(table_name=ti, limit=2500 + 1)

data.sort_index(ascending = True, inplace = True)
data["return"] = data["close"].pct_change() * 100
data = data.dropna()
        #Build the graph
        #value = "Price"
if "Price" == "Volatility":
    data['rolling_6d_volatility'] = data['return'].rolling(window=6).std().dropna()
    fig = px.line(data, x=data.index, y='rolling_6d_volatility', title = f"{ti} 6D Rolling Volatility")
    fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
    fig_dict = fig.to_dict()
            
else:
    fig = px.line(data, x=data.index, y='close', title = f"{ti} Historical Price")
    fig.update_layout(xaxis_title = "Date", yaxis_title = "Price")
    fig_dict = fig.to_dict()

fig = go.Figure(fig_dict)
fig

In [179]:
import plotly.graph_objects as go
import json

url = "http://127.0.0.1:8000/graph"
    # Data to send to path
data = {"ticker": "IBM", "graph_type": "Volatility"}
json_data = json.dumps(data)
    # Response of post request
response = requests.post(url=url, data = json_data, headers={"content-type": "application/json"})
if (
    response.status_code() != 200 and 
    response.headers["content-type"].strip().startswith('application/json')
):
    try:
        response.json()
    except e:
        print(e)


#print(response)
    # Processed response from fastapi
response_json = response.json()
    # Inspect JSON
#print(response_json)
fig_dict = response_json["content"]
#print(fig_dict)
fig = go.Figure(fig_dict)
#fig

TypeError: 'int' object is not callable

In [160]:

url = "http://127.0.0.1:8000/predict"
    # Data to send to path
data = {"ticker": "IBM", "n_days": 5}
json_data = json.dumps(data)
    # Response of post request
response = requests.post(url=url, data = json_data, headers={"Content-Type": "application/json"})
print(response)
    # Processed response from fastapi
response_json = response.json()
    # Inspect JSON
print(response_json)
fig_dict = response_json["content"]
#print(fig_dict)
fig = go.Figure(fig_dict)
fig

<Response [200]>
{'ticker': 'IBM', 'n_days': 5, 'success': 'True', 'forecast': {'2023-01-09T00:00:00': 1.385592896537781, '2023-01-10T00:00:00': 1.3986471664122133, '2023-01-11T00:00:00': 1.410351477429033, '2023-01-12T00:00:00': 1.4208556984451424, '2023-01-13T00:00:00': 1.4302909625669196}, 'message': ''}


KeyError: 'content'

In [155]:
if 2 == 3:
    if 3 == 4:
        print("good")
    else:
        print("Chief Femi")
else: 
    print("femi")

femi


In [168]:
import numpy as np
import plotly.graph_objects as go

x = np.arange(10)

data = (go.Scatter(x=x, y=x**2))
type(data)

plotly.graph_objs._scatter.Scatter

In [170]:
from pydantic.dataclasses import dataclass
from pydantic import BaseModel

@dataclass()
class Foo:
    number: Scatter
    
class Bar(BaseModel):
    number: int
    
f = Foo(fig)
b = Bar(number = 1.4)
print(f)
print(b)

NameError: name 'Scatter' is not defined